In [0]:
dbutils.widgets.text("path", "", "")
path = dbutils.widgets.get("path")

In [0]:
# Raw to Bronce

df_bankloans_raw = (spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .option("sep", ",")
  .option("schema", "string")
  .csv(f"/{path}/bank_loan_medallon_project"))

df_bankloans_raw.show(10)
df_bankloans_raw.printSchema() 
df_bankloans_raw.display() 
df_bankloans_raw.createOrReplaceTempView("bankloans")

# ToDo: guardar csv en bronze (path: /bronce/csv/bank_loan_clean_csv)
df_bankloans_raw.write.format("csv").option("header","true").mode("overwrite").save(f"{path}/bronce/csv")

In [0]:
# cambio de nombre de columnas en raw estructurado (bronce)
df_bankloans_bronce =(spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .option("sep", ",")
  .option("schema", "string")
  .csv(f"/{path}/bronce/csv"))
df_bankloans_bronce = df_bankloans_bronce.withColumnRenamed("ZIP Code", "ZIP_Code")
df_bankloans_bronce= df_bankloans_bronce.withColumnRenamed("Personal Loan", "Personal_Loan")
df_bankloans_bronce= df_bankloans_bronce.withColumnRenamed("Securities Account", "Securities_Account")
df_bankloans_bronce= df_bankloans_bronce.withColumnRenamed("CCAvg" ,"CC_Avg")
df_bankloans_bronce= df_bankloans_bronce.withColumnRenamed("CD Account", "CD_Account")
df_bankloans_bronce = df_bankloans_bronce.withColumnRenamed("income", "Income")

# Raw estructurado (sigue siendo bronce)
df_bankloans_bronce.display()
df_bankloans_bronce.write.format("delta").mode("overwrite").save(f"/{path}/bronce/delta_tables")
spark.read.format("delta").load(f"/{path}/bronce/delta_tables").printSchema()